## Modifying PyPlate to Incorporate Relative Quantities
To incorporate the concept of tags and relative quantities in PyPlate, we need to modify several parts of the PyPlate API. Here are the proposed changes to the docstrings and API behavior:

Substance Class: Modify to include tags.

Transfer Method: Modify to handle relative quantities using tags.

Quantity Validation: Ensure quantities are pgysically reasonable.
1. Modify Substance Class

In [1]:
class Substance:
    """
    Represents a chemical substance with properties and tags.
    
    Attributes:
        name (str): The name of the substance.
        molecular_weight (float): The molecular weight of the substance.
        quantity (float): The quantity of the substance in mmol.
        tags (list of str): List of tags associated with the substance, e.g.,['A', 'ligand'].
    """
    def __init__(self, name, molecular_weight, quantity, tags=None):
        self.name = name
        self.molecular_weight = molecular_weight
        self.quantity = quantity
        self.tags = tags if tags is not None else []

2. Modify Transfer Method from Recipe

In [4]:
def transfer(source, target, quantity):
    """
    Transfers a substance from the source to the target container.

    parametes:
    ----------
    source : Container
        The container to transfer the substance from.
    target : Container
        The container to transfer the substance to.
    quantity : str or float
        The quantity to transfer. If a string, it can specify a relative quantity,
        e.g., '1.1 * A'. The method will parse this and compute the actual quantity
        based on the tag specified.
    
    Raises:
    ------
    ValueError
        If the quantity is not physically reasonable or if the tag is not found.
    """
    if isinstance(quantity, str):
        # Parse relative quntity
        factor, tag = quantity.split('*')
        factor = float(factor.strip())

        # Find the substance with the given tag in the source container
        substance = next((s for s in source.substances if tag.strip() in s.tags), None)
        if not substance:
            raise ValueError(f"Tag {tag.strip()} not found in source container.")

        # Calculate the actual quantity
        quantity = factor * substance.quantity
        
    # Ensure the quantity is physically reasonable
    if quantity <= 0 or quantity > source.remain_volume:
        raise ValueError("Quantity is not physically reasonable.")
    
    # Perform the transfer
    target.add_substance(substance, quantity)
    source.remove_substance(substance, quantity)

3. Quantity Validation

Ensure the quantity transferred is within the physically reasonable limits of the source and target containers.

Docstring Modifications required:

Ensuring Physical Reasonability:
   1. Quantity should be positive: Ensures we are transferring a positive amount.
   2. Quantity should not exceed remaining volume: Ensures we are not transferring more than available.
   3. Quantity should not exceed container capacity: Ensures the target container can hold the transferred substance.

Constraints:
   1. Capacity Constraints: Ensure that the volume of substances in any container does not exceed its maximum capacity.
   2. Tag Constraints: Ensure that each tag uniquely identifies a substance or a group of substances.

This approach keeps the API changes minimal while providing flexibility for users to specify relative quantities in their recipes. The validation ensures that the operations remain within practical and safe limits.
